## 交互式多模态检索 

提供以图搜图和以文本搜图两种检索方式

以图搜图: 输入图片 url, 返回与输入图片相似的图片
以文本搜图: 输入文本, 返回与输入文本相似的图片

In [ ]:
import io
import ipywidgets as widgets
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import display, clear_output
from src.lindorm import Lindorm

lindorm = Lindorm()


# 创建输入框和按钮
text_input = widgets.Text(
    value='',
    placeholder='输入关键字',
    description='以文搜图:',
    disabled=False
)

file_input = widgets.FileUpload(
    accept='image/*',
    description='以图搜图',
    multiple=False  # 只允许选择一张图片
)

# 定义搜索按钮
search_buttons = [
    widgets.Button(description="纯向量检索", button_style=''),
]

output = widgets.Output()

def show_hits(hits):
    if hits is None or len(hits) == 0:
        print("图片搜索失败")
        return
    print('search, count', len(hits))
    for hit in hits:
        print(hit.get('_id'), hit.get('_score'))
    # 创建一个 3x3 的子图
    fig, axes = plt.subplots(3, 3, figsize=(9, 9))
    # 遍历图片 URL
    for ax, hit in zip(axes.flatten(), hits):
        url = hit.get('_id')
        # 在子图中显示图片
        img = Image.open(url)
        # print(url, img)
        ax.imshow(img)
        ax.axis('off')  # 隐藏坐标轴
    plt.tight_layout()  # 调整布局
    plt.show()

# 定义按钮点击事件处理函数
def on_button_clicked(b):
    with output:
        clear_output()  # 清除上次输出
        if text_input.value:
            code, embedding = lindorm.text_embedding(text_input.value)
        else:
            print("请先输入描述文字")
            return
        
        if b.description == "纯向量检索":
            hits = lindorm.knn_search(embedding)
        elif b.description == "RRF融合检索":
            if text_input.value == '':
                print("请先输入关键字")
                return
            hits = lindorm.rrf_search(text_input.value, embedding)
        print("文本搜图")
        show_hits(hits)
        
# 定义图片显示函数
def show_uploaded_image(change):
    with output:
        clear_output()  # 清除上次输出
        if change['new']:
            # 获取上传的文件内容
            uploaded_file = change['new'][0]
            content = uploaded_file['content']
            # 使用 PIL 打开图片
            image = Image.open(io.BytesIO(content))
            # 使用 matplotlib 显示图片
            plt.imshow(image)
            plt.axis('off')  # 不显示坐标轴
            plt.show()
            code, embedding = lindorm.picture_embedding(content)
            hits = lindorm.knn_search(embedding)
            print("图片上传检索")
            show_hits(hits)
            
        
for button in search_buttons:
    button.on_click(lambda b: on_button_clicked(b))
    
# 绑定文件上传事件
file_input.observe(show_uploaded_image, names='value')

display(file_input, text_input, *search_buttons, output)
